In [1]:
import pandas as pd
import os
import numpy as np
import time
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv("C://Users//User//Desktop//NUS//Year 3 Sem 1//DSA3101//DSA3101project//cleaned_data//imputed_data.csv", parse_dates=[1], names=['Panel ID', 'Date', 'Category', 'Pack Size', 'Volume', 'Spend'], skiprows=1)
panel_demo = pd.read_excel("C://Users//User//Desktop//NUS//Year 3 Sem 1//DSA3101//DSA3101project//data//DSA3101_Hackathon_Panelists_Demographics.xlsx")
panel_demo.columns = ["ID", "BMI", "Income", "Ethnicity", "Lifestage", "Stata", "HH", "Location"]

data.sort_values("Date", inplace=True)

le = preprocessing.LabelEncoder()
data['Week'] = le.fit_transform(data.Date)

data = data[["Panel ID", "Category", "Spend", "Week"]].merge(data.groupby(["Panel ID"]).agg({"Week": min}).reset_index(), on="Panel ID")
data.columns = ["Panel ID", "Category", "Spend", "Week", "First Week"]
data.tail()

,Panel ID,Category,Spend,Week,First Week
1300582,Panel 801865301,Cordials,3.0,147,136
1300583,Panel 801865301,Liquid Milk,4.2,147,136
1300584,Panel 801865301,Condensed/Evap Milk,5.1,147,136
1300585,Panel 801865301,Milk Powder-Adult,17.5,147,136
1300586,Panel 801865301,Chocolate,8.7,147,136


In [2]:
train = data[(data["First Week"] < 52) & (data["Week"] < 52)]    # 2575 customers
val = data[( 52 <= data["First Week"]) & (data["First Week"] < 104) & (52 <= data["Week"]) & (data["Week"] < 104)]    # 373 customers
test = data[(data["First Week"] >= 104) & (data["Week"] >= 104)]    # 288 customers
print(train["Panel ID"].unique().shape, val["Panel ID"].unique().shape, test["Panel ID"].unique().shape)

(2575,) (373,) (288,)


In [3]:
train.head()

,Panel ID,Category,Spend,Week,First Week
0,Panel 317007101,Frozen Food,5.8,0,0
1,Panel 317007101,Cooking Sauces,8.5,0,0
2,Panel 317007101,Snack,8.0,0,0
3,Panel 317007101,Snack,5.0,0,0
4,Panel 317007101,Cake,6.0,0,0


In [4]:
pd.__version__

'1.0.1'

In [5]:
! conda install pandas

Solving environment: ...working... done

# All requested packages already installed.



In [11]:
%%time
def week_EMA(x):
    lst = x.shift(-1)-x
    return lst.ewm(alpha=1e-3).mean().tail(1).values

def Monetary_EMA(x):
    return x.ewm(alpha=1e-3).mean().tail(1).values

def frequency(data, cutoff_date):
    """
    cutoff_date = 52, 104 or 156
    """
    df = data.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index()

    df_last_week = data[["Panel ID"]].drop_duplicates()
    df_last_week["Week"] = cutoff_date - 1

    df = df.merge(df_last_week, on=["Panel ID", "Week"], how="outer") \
        .fillna(0) \
        .groupby("Panel ID") \
        .agg({"Week": [min, max, week_EMA]}) \
        .reset_index()
    
    df.columns = ["Panel ID", "First Week", "Last Week", "Frequency_EMA"]
    df = df[df["First Week"] <= cutoff_date-3]    # remove customers that newly joined close to our cutoff date
    return df[["Panel ID", "Frequency_EMA"]]

def monetary(data, cutoff_date):
    """
    cutoff_date = 52, 104 or 156
    """
    df = data.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index() \
        .pivot_table(index=['Panel ID'], columns='Week', values='Spend') \
        .fillna(0) \
        .reset_index() \
        .melt(id_vars=["Panel ID"], value_vars=list(range(cutoff_date-52, cutoff_date)))

    df = df.merge(data[["Panel ID", "First Week"]], on="Panel ID", how="left")
    df = df[df["First Week"] <= df["Week"]]
    df = df.groupby("Panel ID") \
        .agg({"value": Monetary_EMA}) \
        .reset_index()

    df.columns = ["Panel ID", "Monetary_EMA"]
    return df

def first_purchases(df):
    pivot = data.pivot_table(index=['Panel ID', "First Week"], columns='Category', values='Spend')
    df = df[["Panel ID", "First Week"]] \
        .merge(data[["Panel ID", "Week", "Category", "Spend"]], how="left", left_on=["Panel ID", "First Week"], right_on=["Panel ID", "Week"]) \
        .pivot_table(index=['Panel ID', 'First Week'], columns='Category', values='Spend') \
        .reset_index()
    df = pd.concat([df, pd.DataFrame(columns=list(pivot.columns[~pivot.columns.isin(df.columns)]))]) \
        .fillna(0) \
        .drop(['First Week'], axis=1)
    return df

train_df = frequency(train, cutoff_date=52) \
    .merge(monetary(train, cutoff_date=52), on="Panel ID", how="left") \
    .merge(first_purchases(train), on=["Panel ID"], how="left") \
    .merge(panel_demo, how="left", left_on="Panel ID", right_on="ID") \
    .drop(['ID'], axis=1)
val_df = frequency(val, cutoff_date=104) \
    .merge(monetary(val, cutoff_date=104), on="Panel ID", how="left") \
    .merge(first_purchases(val), on=["Panel ID"], how="left") \
    .merge(panel_demo, how="left", left_on="Panel ID", right_on="ID") \
    .drop(['ID'], axis=1)
test_df = frequency(test, cutoff_date=156) \
    .merge(monetary(test, cutoff_date=156), on="Panel ID", how="left") \
    .merge(first_purchases(test), on=["Panel ID"], how="left") \
    .merge(panel_demo, how="left", left_on="Panel ID", right_on="ID") \
    .drop(['ID'], axis=1)

print(train_df.shape, val_df.shape, test_df.shape)

# train_df.to_csv("/content/drive/My Drive/data/train_df.csv", index=False)
# test_df.to_csv("/content/drive/My Drive/data/test_df.csv", index=False)
# val_df.to_csv("/content/drive/My Drive/data/val_df.csv", index=False)

ValueError: Must produce aggregated value

In [12]:
%%time
def week_EMA(x):
    lst = x.shift(-1)-x
    return lst.ewm(alpha=1e-3).mean().tail(1).values

def Monetary_EMA(x):
    return x.ewm(alpha=1e-3).mean().tail(1).values

def frequency(data, cutoff_date):
    """
    cutoff_date = 52, 104 or 156
    """
    df = data.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index()

    df_last_week = data[["Panel ID"]].drop_duplicates()
    df_last_week["Week"] = cutoff_date - 1

    df = df.merge(df_last_week, on=["Panel ID", "Week"], how="outer") \
        .fillna(0) \
        .groupby("Panel ID") \
        .agg({"Week": [min, max, week_EMA]}) \
        .reset_index()
    
    df.columns = ["Panel ID", "First Week", "Last Week", "Frequency_EMA"]
    df = df[df["First Week"] <= cutoff_date-3]    # remove customers that newly joined close to our cutoff date
    return df[["Panel ID", "Frequency_EMA"]]

def monetary(data, cutoff_date):
    """
    cutoff_date = 52, 104 or 156
    """
    df = data.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index() \
        .pivot_table(index=['Panel ID'], columns='Week', values='Spend') \
        .fillna(0) \
        .reset_index() \
        .melt(id_vars=["Panel ID"], value_vars=list(range(cutoff_date-52, cutoff_date)))

    df = df.merge(data[["Panel ID", "First Week"]], on="Panel ID", how="left")
    df = df[df["First Week"] <= df["Week"]]
    df = df.groupby("Panel ID") \
        .agg({"value": Monetary_EMA}) \
        .reset_index()

    df.columns = ["Panel ID", "Monetary_EMA"]
    return df

def first_purchases(df):
    pivot = data.pivot_table(index=['Panel ID', "First Week"], columns='Category', values='Spend')
    df = df[["Panel ID", "First Week"]] \
        .merge(data[["Panel ID", "Week", "Category", "Spend"]], how="left", left_on=["Panel ID", "First Week"], right_on=["Panel ID", "Week"]) \
        .pivot_table(index=['Panel ID', 'First Week'], columns='Category', values='Spend') \
        .reset_index()
    df = pd.concat([df, pd.DataFrame(columns=list(pivot.columns[~pivot.columns.isin(df.columns)]))]) \
        .fillna(0) \
        .drop(['First Week'], axis=1)
    return df

train_df = frequency(train, cutoff_date=52)

ValueError: Must produce aggregated value

In [34]:
def week_EMA(x):
    lst = x.shift(-1)-x
    return lst.ewm(alpha=1e-3).mean().tail(1).values

cutoff_date = 52

df = train.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index()

df_last_week = train[["Panel ID"]].drop_duplicates()
df_last_week["Week"] = cutoff_date - 1

df = df.merge(df_last_week, on=["Panel ID", "Week"], how="outer") \
        .fillna(0) \
        .groupby("Panel ID") \
        .agg({"Week": [min, max, week_EMA]}) \
        .reset_index()
    
df.columns = ["Panel ID", "First Week", "Last Week", "Frequency_EMA"]
df = df[df["First Week"] <= cutoff_date-3]  


KeyError: 'Method'

In [46]:
df = train.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index()

df_last_week = train[["Panel ID"]].drop_duplicates()
df_last_week["Week"] = cutoff_date - 1

df = df.merge(df_last_week, on=["Panel ID", "Week"], how="outer") \
        .fillna(0) \
        .groupby("Panel ID") \
        .agg({"Week": [min, max, week_EMA]}) \
        .reset_index()

df

ValueError: Must produce aggregated value

In [9]:
train_df.Frequency_EMA.value_counts(normalize=True, bins=range(52))
# ~76.8% of the panels in training have Frequency_EMA <= 2 so we can consider a customer to be frequent if the Frequency_EMA <= 2

NameError: name 'train_df' is not defined

In [6]:
train_df.Monetary_EMA.value_counts(normalize=True, bins=range(0, int(train_df.Monetary_EMA.max())+1, 40))
# ~70.6% of the panels in training have Monetary_EMA <= 40 so we can consider a customer to be frequent if the Monetary_EMA <= 40

(-0.001, 40.0]    0.705631
(40.0, 80.0]      0.238447
(80.0, 120.0]     0.040000
(120.0, 160.0]    0.009709
(160.0, 200.0]    0.003495
(200.0, 240.0]    0.001942
(240.0, 280.0]    0.000388
Name: Monetary_EMA, dtype: float64

# Modeling

In [7]:
scaler = None
def dataloader(df, scalar, target, scale=True):
    columns = list(panel_demo.columns)
    columns.remove("ID")
    data_df = df.copy()
    data_df["Frequency_EMA"] = (data_df["Frequency_EMA"] <= 2).map(int)
    data_df["Monetary_EMA"] = (data_df["Monetary_EMA"] > 40).map(int)
    data_df = pd.get_dummies(data_df, columns=columns)
    data = data_df.drop(['Panel ID', 'Frequency_EMA', 'Monetary_EMA'], axis=1)
    if scale:
        if scaler == None:
            X = pd.DataFrame(MinMaxScaler().fit_transform(data), 
                                columns=data.columns)
        else:
            X = pd.DataFrame(scaler.transform(data), 
                                columns=data.columns)
    else:
        X = data.copy()
    y = data_df[target]
    data = pd.concat([X, y], axis=1)
    return X, y, data, scalar
    
# train_X, train_y, train, scaler = dataloader(train_df, scaler, target="Frequency_EMA")
# val_X, val_y, val, scaler = dataloader(val_df, scaler, target="Frequency_EMA")
# test_X, test_y, test, scaler = dataloader(test_df, scaler, target="Frequency_EMA")
train_X, train_y, train, scaler = dataloader(train_df, scaler, target="Monetary_EMA")
val_X, val_y, val, scaler = dataloader(val_df, scaler, target="Monetary_EMA")
test_X, test_y, test, scaler = dataloader(test_df, scaler, target="Monetary_EMA")